In [1]:
# importing the packages

import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt 
%matplotlib inline
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, BaggingClassifier, AdaBoostClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix, accuracy_score, f1_score
from sklearn.model_selection import StratifiedKFold, KFold, train_test_split

In [2]:
train = pd.read_csv('insure1_train.csv')
test = pd.read_csv('insure1_test.csv')

In [3]:
train.head()

,EmployeeNo,Gender,Trainings_Attended,Year_of_birth,Last_performance_score,Year_of_recruitment,Targets_met,Previous_Award,Training_score_average,Foreign_schooled,Past_Disciplinary_Action,Previous_IntraDepartmental_Movement,Promoted_or_Not,divisional,statea,marital
0,YAK/S/00001,0,2,1986,12.5,2011,1,0,41,0,0,0,0,2,0,2
1,YAK/S/00002,1,2,1991,12.5,2015,0,0,52,1,0,0,0,2,1,12
2,YAK/S/00003,1,2,1987,7.5,2012,0,0,42,1,0,0,0,2,1,12
3,YAK/S/00004,1,3,1982,2.5,2009,0,0,42,1,0,0,0,3,1,12
4,YAK/S/00006,1,3,1990,7.5,2012,0,0,77,1,0,0,0,3,1,12


In [4]:
train.columns

Index(['EmployeeNo', 'Gender', 'Trainings_Attended', 'Year_of_birth',
       'Last_performance_score', 'Year_of_recruitment', 'Targets_met',
       'Previous_Award', 'Training_score_average', 'Foreign_schooled',
       'Past_Disciplinary_Action', 'Previous_IntraDepartmental_Movement',
       'Promoted_or_Not', 'divisional', 'statea', 'marital'],
      dtype='object')

In [5]:
test_id = test['EmployeeNo']

In [6]:

train = train.drop('EmployeeNo', axis=1)
test = test.drop('EmployeeNo', axis=1)

In [7]:

X = train.drop('Promoted_or_Not', axis=1)
y = train.Promoted_or_Not

In [8]:
# calling in the train_test_split function

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

In [9]:
from sklearn.metrics import auc

In [10]:
#declaring our model

model=CatBoostClassifier(n_estimators=800,eval_metric='AUC',max_depth=5,learning_rate=0.1,od_wait=50, 
                              subsample=0.9,bootstrap_type='Bernoulli',metric_period=20,
                     #l2_leaf_reg=5,#bagging_temperature=0.85,random_strength=100,
                     use_best_model=True)


In [11]:

model.fit(X_train, y_train, eval_set=(X_test, y_test))

0:	test: 0.5978666	best: 0.5978666 (0)	total: 259ms	remaining: 3m 26s
20:	test: 0.8279235	best: 0.8296027 (18)	total: 852ms	remaining: 31.6s
40:	test: 0.8358623	best: 0.8358623 (40)	total: 1.37s	remaining: 25.4s
60:	test: 0.8408584	best: 0.8408584 (60)	total: 1.83s	remaining: 22.2s
80:	test: 0.8410385	best: 0.8417327 (75)	total: 2.27s	remaining: 20.1s
100:	test: 0.8412635	best: 0.8417327 (75)	total: 2.63s	remaining: 18.2s
120:	test: 0.8421949	best: 0.8423207 (119)	total: 2.99s	remaining: 16.8s
140:	test: 0.8434458	best: 0.8434458 (140)	total: 3.33s	remaining: 15.6s
160:	test: 0.8441324	best: 0.8442107 (158)	total: 3.66s	remaining: 14.5s
180:	test: 0.8445982	best: 0.8446169 (179)	total: 3.97s	remaining: 13.6s
200:	test: 0.8444117	best: 0.8449007 (181)	total: 4.26s	remaining: 12.7s
220:	test: 0.8444102	best: 0.8449007 (181)	total: 4.56s	remaining: 12s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.8449007252
bestIteration = 181

Shrink model to first 182 iterations.


In [12]:
pred = model.predict_proba(test)[:, 1]

In [13]:
#calling in the train_test_split function

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

In [14]:
# first tuning using the StratifedKfold and Catboost model algorithm

from sklearn.metrics import accuracy_score,confusion_matrix,roc_auc_score
errcb=[]
y_pred_totcb=[]
from sklearn.model_selection import KFold,StratifiedKFold, TimeSeriesSplit, GroupKFold
fold=StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
i=1
for train_index, test_index in fold.split(X,y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    m=CatBoostClassifier(n_estimators=800,eval_metric='AUC',max_depth=4,learning_rate=0.1,#reg_lambda=5,#5
                              subsample=0.9,bootstrap_type='Bernoulli',#leaf_estimation_iterations=10,
                    #l2_leaf_reg=5,#bagging_temperature=0.85,random_strength=100,
                     use_best_model=True)
    #CatBoostClassifier(n_estimators=1000,eval_metric='AUC',max_depth=5,learning_rate=0.1,reg_lambda=5,#5
                              #subsample=0.9,bootstrap_type='Bernoulli',
                    #l2_leaf_reg=5,#bagging_temperature=0.85,random_strength=100,
                     #use_best_model=True)
    m.fit(X_train,y_train,eval_set=[(X_train,y_train),(X_test, y_test)], early_stopping_rounds=100,verbose=100)
    preds=m.predict_proba(X_test)[:, 1]
    print("err: ",roc_auc_score(y_test,preds))
    errcb.append(roc_auc_score(y_test,preds))
    p = m.predict_proba(test)[:, 1]
    y_pred_totcb.append(p)

0:	test: 0.7580348	test1: 0.7433860	best: 0.7433860 (0)	total: 31.2ms	remaining: 25s
100:	test: 0.8518502	test1: 0.8395432	best: 0.8396371 (96)	total: 2.37s	remaining: 16.4s
200:	test: 0.8627687	test1: 0.8406525	best: 0.8411309 (166)	total: 5.03s	remaining: 15s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.8411308794
bestIteration = 166

Shrink model to first 167 iterations.
err:  0.84113087940391
0:	test: 0.7563391	test1: 0.7636168	best: 0.7636168 (0)	total: 39.7ms	remaining: 31.7s
100:	test: 0.8525894	test1: 0.8326561	best: 0.8327564 (99)	total: 2.31s	remaining: 16s
200:	test: 0.8630827	test1: 0.8367112	best: 0.8367112 (200)	total: 3.98s	remaining: 11.9s
300:	test: 0.8709800	test1: 0.8374807	best: 0.8375955 (288)	total: 5.44s	remaining: 9.01s
400:	test: 0.8779019	test1: 0.8372610	best: 0.8377670 (342)	total: 6.8s	remaining: 6.77s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.8377670455
bestIteration = 342

Shrink model to first 343 iterat

In [15]:
np.mean(errcb)

0.8414101052362994

In [16]:
# second tuning using the StratifedKfold and Catboost model algorithm

from sklearn.metrics import accuracy_score,confusion_matrix,roc_auc_score
errcb1=[]
y_pred_totcb1=[]
from sklearn.model_selection import KFold,StratifiedKFold, TimeSeriesSplit
fold=StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
i=1
for train_index, test_index in fold.split(X,y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    m1=CatBoostClassifier(n_estimators=1000,eval_metric='AUC',max_depth=4,learning_rate=0.1,od_wait=50, reg_lambda=3,
                              bootstrap_type='Bayesian',
                     use_best_model=True)
    m1.fit(X_train,y_train,eval_set=[(X_train,y_train),(X_test, y_test)], early_stopping_rounds=100,verbose=100)
    preds=m1.predict_proba(X_test)[:, 1]
    print("err: ",roc_auc_score(y_test,preds))
    errcb1.append(roc_auc_score(y_test,preds))
    p1 = m1.predict_proba(test)[:, 1]
    y_pred_totcb1.append(p1)


0:	test: 0.7639707	test1: 0.7526133	best: 0.7526133 (0)	total: 30ms	remaining: 30s
100:	test: 0.8511388	test1: 0.8402999	best: 0.8403038 (99)	total: 3.09s	remaining: 27.5s
200:	test: 0.8602711	test1: 0.8406891	best: 0.8408316 (122)	total: 5.05s	remaining: 20.1s
300:	test: 0.8666584	test1: 0.8409935	best: 0.8413950 (261)	total: 6.61s	remaining: 15.4s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.8413949714
bestIteration = 261

Shrink model to first 262 iterations.
err:  0.8413949714405192
0:	test: 0.7626894	test1: 0.7555222	best: 0.7555222 (0)	total: 11ms	remaining: 11s
100:	test: 0.8525941	test1: 0.8349135	best: 0.8349135 (100)	total: 1.6s	remaining: 14.3s
200:	test: 0.8616060	test1: 0.8361940	best: 0.8363691 (196)	total: 3.18s	remaining: 12.6s
300:	test: 0.8682860	test1: 0.8372142	best: 0.8372667 (276)	total: 4.71s	remaining: 10.9s
400:	test: 0.8733520	test1: 0.8378610	best: 0.8378610 (400)	total: 6.01s	remaining: 8.97s
500:	test: 0.8781594	test1: 0.8377695	best

In [17]:
np.mean(errcb1)

0.8415397590792294

In [18]:
# third tuning using the Kfold and XGboost model algorithm

from sklearn.metrics import accuracy_score,confusion_matrix,roc_auc_score
errcb2=[]
y_pred_totcb2=[]
from sklearn.model_selection import KFold,StratifiedKFold, TimeSeriesSplit
fold=KFold(n_splits=5)
i=1
for train_index, test_index in fold.split(X,y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    m2=XGBClassifier(max_depth=6, learning_rate=0.1, n_estimators=800, silent=True, metrics='auc',colsample_bylevel=0.8, reg_alpha=0.8)
    m2.fit(X_train,y_train,eval_set=[(X_train,y_train),(X_test, y_test)], early_stopping_rounds=100,verbose=100)
    preds=m2.predict_proba(X_test)[:, 1]
    print("err: ",roc_auc_score(y_test,preds))
    errcb2.append(roc_auc_score(y_test,preds))
    p2 = m2.predict_proba(test)[:, 1]
    y_pred_totcb2.append(p2)

[07:20:25] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { metrics, silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	validation_0-error:0.07286	validation_1-error:0.07530
Multiple eval metrics have been passed: 'validation_1-error' will be used for early stopping.

Will train until validation_1-error hasn't improved in 100 rounds.
[100]	validation_0-error:0.06972	validation_1-error:0.07556
Stopping. Best iteration:
[18]	validation_0-error:0.07188	validation_1-error:0.07464

err:  0.833891762347213
[07:20:28] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { metrics, silent } might not be used.

  This may not be accurate due to some parameters are only used

In [19]:
#checking the np.mean(errcb) of the third tuning

np.mean(errcb2)

0.8360873492318079

In [21]:

d = {"EmployeeNo": test_id, 'Promoted_or_Not': np.mean(y_pred_totcb, 0)}
test_predictions = pd.DataFrame(data=d)
test_predictions = test_predictions[["EmployeeNo", 'Promoted_or_Not']]

In [22]:
test_predictions.to_csv('m1.csv', index=False)

In [23]:
test_predictions.head()

,EmployeeNo,Promoted_or_Not
0,YAK/S/00005,0.057303
1,YAK/S/00011,0.050939
2,YAK/S/00015,0.031044
3,YAK/S/00016,0.005406
4,YAK/S/00017,0.159883


In [24]:
d = {"EmployeeNo": test_id, 'Promoted_or_Not': np.mean(y_pred_totcb1, 0)}
test_predictioned = pd.DataFrame(data=d)
test_predictioned = test_predictioned[["EmployeeNo", 'Promoted_or_Not']]

In [25]:
test_predictioned.to_csv('m2.csv', index=False)

In [26]:
test_predictioned.head()

,EmployeeNo,Promoted_or_Not
0,YAK/S/00005,0.067346
1,YAK/S/00011,0.056261
2,YAK/S/00015,0.031907
3,YAK/S/00016,0.005786
4,YAK/S/00017,0.155454


In [27]:
d = {"EmployeeNo": test_id, 'Promoted_or_Not': np.mean(y_pred_totcb2, 0)}
test_prediction = pd.DataFrame(data=d)
test_prediction = test_prediction[["EmployeeNo", 'Promoted_or_Not']]

In [28]:
test_prediction.to_csv('m3.csv', index=False)

In [30]:
test_prediction.head()

,EmployeeNo,Promoted_or_Not
0,YAK/S/00005,0.117810
1,YAK/S/00011,0.123584
2,YAK/S/00015,0.114467
3,YAK/S/00016,0.094378
4,YAK/S/00017,0.215146


In [31]:
a = pd.read_csv('m1.csv')
b = pd.read_csv('m2.csv')
c = pd.read_csv('m3.csv')

In [33]:
# correlating the variable of my tuned model
a['Promoted_or_Not'].corr(b['Promoted_or_Not']), a['Promoted_or_Not'].corr(c['Promoted_or_Not']),b['Promoted_or_Not'].corr(c['Promoted_or_Not'])

(0.9975736011348607, 0.9869472224956242, 0.986556284911308)

In [34]:
# correlating the blended prediction of my tuned model
test_predictions['Promoted_or_Not'].corr(test_predictioned['Promoted_or_Not']), test_predictions['Promoted_or_Not'].corr(test_prediction['Promoted_or_Not']), test_predictioned['Promoted_or_Not'].corr(test_prediction['Promoted_or_Not'])

(0.9975736011348608, 0.9869472225733122, 0.9865562850309286)

In [35]:
# blending the whole models into test_predictions model
test_predictions['Promoted_or_Not'] = (a['Promoted_or_Not'] * 0.69 + b['Promoted_or_Not'] * 0.35 + c['Promoted_or_Not'] * 0.28).round(2)

In [36]:
test_predictions.to_csv('insure_stack1.csv', index=False)

In [37]:
train.to_csv('insure2_train.csv', index=False)
test.to_csv('insure2_test.csv', index=False)